In [59]:
import pandas as pd
import cv2
import numpy as np
import matplotlib.pyplot as plt
#jupyter内で画像を表示
%matplotlib inline 
#別ウィンドウで画像表示
# %matplotlib qt 
import csv
import json
from ipywidgets import interact, interactive, fixed, RadioButtons
import ipywidgets as widgets
from IPython.display import display
import pathlib
import glob
import re
import copy

In [67]:
shadow = '16.500000_1.400000'
InterPoint = '0.750000'
evaluation_path = "../../get_data/" + shadow + "/" + InterPoint
sigo_file_list = []
# sigo_files = glob.glob("../../ns3.30/data/sigo/*")
sigo_files = glob.glob(evaluation_path + "/sigo/*")
for file in sigo_files:
    sigo_file_list.append(file)
    
send_sigo_file_list = []
# send_sigo_files = glob.glob("../../ns3.30/data/send_sigo/*")
send_sigo_files = glob.glob(evaluation_path + "/send_sigo/*")
for file in send_sigo_files:
    send_sigo_file_list.append(file)
print('receive log')
print(sigo_file_list)
print('send log')
print(send_sigo_file_list)

receive log
[]
send log
[]


In [68]:
seed_list = []
nodenum_list = ['200', '300', '400'] ###############nodenumの形式を変更したらここはベタ打ちで変更を加える
regex = re.compile('\d+')

for path in sigo_file_list:
    for line in path.splitlines():
        match = regex.findall(line)
        if match[6] not in seed_list:
            seed_list.append(match[6])
            
print(seed_list)
print('simulation 試行回数',len(seed_list))

[]
simulation 試行回数 0


In [69]:
sigo_dict = {}
check_seed = seed_list[0]
sigo_dict[check_seed] = {}
for path in sigo_file_list:
    for line in path.splitlines():
      match = regex.findall(line) # match[2] = seed ,match[3] = nudenum
     
    if match[6] == check_seed:
        sigo_dict[match[6]][match[7]] = path
    else:#ロープ中seed値が変わるとき
        sigo_dict[match[6]] = {}
        sigo_dict[match[6]][match[7]] = path
        check_seed = match[6] #seed値の更新
        
send_sigo_dict = {}
check_seed = seed_list[0]
send_sigo_dict[check_seed] = {}
for path in send_sigo_file_list:
    for line in path.splitlines():
      match = regex.findall(line) # match[2] = seed ,match[3] = nudenum
     
    if match[6] == check_seed:
        send_sigo_dict[match[6]][match[7]] = path
    else:#ロープ中seed値が変わるとき
        send_sigo_dict[match[6]] = {}
        send_sigo_dict[match[6]][match[7]] = path
        check_seed = match[6] #seed値の更新
        

print(sigo_dict)
print(send_sigo_dict)

IndexError: list index out of range

In [63]:
def packet_ratio(path):
    #csvファイルの取得
    path_df = pd.read_csv(path)
    dest_list = path_df['destination_id'].value_counts().index.tolist() #重複なしのdestination_idを格納
    #目的地に届いたidだけをlistに保存
    reach_list = path_df[path_df['recv_id'] == path_df['destination_id']]['destination_id'].value_counts().index.tolist()
#     PDR = len(reach_list)/len(dest_list)
    PDR = len(reach_list)/10
    
#     *************** len(dest_list): destination_idのリストをいったん10個で定義
    return PDR

def packet_hop(path):
    path_df = pd.read_csv(path)
    dest_list = path_df['destination_id'].value_counts().index.tolist() #重複なしのdestination_idを格納
    #目的地に届いたidだけをlistに保存
    reach_list = path_df[path_df['recv_id'] == path_df['destination_id']]['destination_id'] 
    #reach_listを回す
    total_hop = 0
    for reach_id in reach_list:
        hop = path_df[(path_df['destination_id'] == reach_id) & (path_df['destination_id'] == path_df['recv_id'])]['hopcount']
        hop = int(hop.head(1).iloc[-1])
        total_hop += hop
    if(len(reach_list) != 0):
        average_hop = total_hop / len(reach_list)
    else:
        average_hop = total_hop
    
    return average_hop

def packet_delay(path):
    delay_list = []
    #csvファイルの取得
    path_df = pd.read_csv(path)
    #目的地に届いたidだけをlistに保存
    reach_list = path_df[path_df['recv_id'] == path_df['destination_id']]['destination_id'].value_counts().index.tolist()
    #reach_listを回す
    if (len(reach_list) == 0):
        delay = 0
        print('reach_list none')
        return delay
    for reach_id in reach_list:
        source_time = path_df[path_df['destination_id'] == reach_id]['time']
        recv_time = path_df[(path_df['destination_id'] == reach_id) & (path_df['destination_id'] == path_df['recv_id'])]['time']
        delay = int(recv_time.head(1).iloc[-1]) - int(source_time.head(1).iloc[-1])
        delay = delay / 1000000
        delay_list.append(delay)
    delay_mean = np.mean(delay_list)
    return delay_mean

def packet_over_head_send(send_path, recv_path):
    send_path_df = pd.read_csv(send_path)
    send_column_num = len(send_path_df.columns) #カラム数を取得
#     display(path_df.iloc[:,send_column_num-1]) 末尾の取得方法　path_df = pd.read_csv(recv_path)末尾=送信ログ(送信したかどうか)
    #目的地に届いたidだけをlistに保存
    recv_path_df = pd.read_csv(recv_path)
    reach_list = recv_path_df[recv_path_df['recv_id'] == recv_path_df['destination_id']]['destination_id'].value_counts().index.tolist()
    send_count = send_path_df[send_path_df.iloc[:,send_column_num-1] == 1].value_counts() #最後のカラムの値が1
#     print(len(send_count))
    if(len(reach_list) != 0):
        overhead = len(send_count)/len(reach_list)
    else:
        overhead = len(send_count)
    return overhead

In [64]:
def col_average(seeds):
    new_list = []
    for seed in seeds:
        del seed[len(nodenum_list)] #1列ずつ平均を算出するうえで邪魔なseed値を消していく seedの個数 
        new_list.append(seed)
#     np.set_printoptions(precision=3, suppress=True)
    np_new_list = np.array(new_list)
    return np.mean(np_new_list, axis=0)

In [65]:
def header(protocol, writer, index):
    writer.writerow([])
    writer.writerow([protocol, index])
    writer.writerow(['200','300','400','seed'])

In [66]:
def write(seeds):
    for seed in seeds:
        writer.writerow(seed)

with open(evaluation_path + '/Performance_evaluation_5_30.csv', 'w', newline='') as csv_file:
    #---------------------------PDR---------------------------
    sigo_pdr_average_list = []
    sigo_delay_average_list = []
    sigo_overhead_average_list = []
    sigo_hop_average_list = []
    
    #headerを設定
    writer = csv.writer(csv_file)
    header('SIGO', writer, 'PDR')
    #--sigo--#
    for seed in seed_list:
        sigo_pdr_ratio_list = []
        for node in nodenum_list:
            path = sigo_dict[seed][node]
            ratio = packet_ratio(path)
            sigo_pdr_ratio_list.append(ratio)
        sigo_pdr_ratio_list.append(seed)
        writer.writerow(sigo_pdr_ratio_list)
        sigo_pdr_average_list.append(sigo_pdr_ratio_list)
    #------------------------delay-----------------------
    header('SIGO', writer, 'delay')
    #--sigo--#
    for seed in seed_list:
        sigo_delay_list = []
        for node in nodenum_list:
            path = sigo_dict[seed][node]
            delay = packet_delay(path)
            sigo_delay_list.append(delay)
        sigo_delay_list.append(seed)
        writer.writerow(sigo_delay_list)
        sigo_delay_average_list.append(sigo_delay_list)    
    #------------------------overhead-----------------------
    header('SIGO', writer, 'Overhead')
    #--sigo--#
    for seed in seed_list:
        sigo_overhead_list = []
        for node in nodenum_list:
            recv_path = sigo_dict[seed][node]
            print(seed)
            send_path = send_sigo_dict[seed][node]
#             print('path', 'seed', seed)
            overhead = packet_over_head_send(send_path, recv_path)
            sigo_overhead_list.append(overhead)
        sigo_overhead_list.append(seed)
        writer.writerow(sigo_overhead_list)
        sigo_overhead_average_list.append(sigo_overhead_list)
    #------------------------Hop-----------------------
    header('SIGO', writer, 'Hop')
    #--sigo--#
    for seed in seed_list:
        sigo_hop_list = []
        for node in nodenum_list:
            path = sigo_dict[seed][node]
            hop = packet_hop(path)
            sigo_hop_list.append(hop)
        sigo_hop_list.append(seed)
        writer.writerow(sigo_hop_list)
        sigo_hop_average_list.append(sigo_hop_list)
        
    header('SIGO', writer , 'PDR-average')
    writer.writerow(col_average(sigo_pdr_average_list))
    header('SIGO', writer , 'delay-average')
    writer.writerow(col_average(sigo_delay_average_list))
    header('SIGO', writer , 'overhead-average')
    writer.writerow(col_average(sigo_overhead_average_list))
    header('SIGO', writer , 'hop-average')
    writer.writerow(col_average(sigo_hop_average_list))
    print('finish')

10001
10001
10001
10002
10002
10002
10003
10003
10003
10004
10004
10004
10005
10005
10005
10006
10006
10006
10007
10007
10007
10008
10008
10008
10009
10009
10009
10010
10010
10010
10011
10011
10011
10012
10012
10012
10013
10013
10013
10014
10014
10014
10015
10015
10015
10016
10016
10016
10017
10017
10017
10018
10018
10018
10019
10019
10019
10020
10020
10020
10021
10021
10021
10022
10022
10022
10023
10023
10023
10024
10024
10024
10025
10025
10025
10026
10026
10026
10027
10027
10027
10028
10028
10028
10029
10029
10029
10030
10030
10030
10031
10031
10031
10032
10032
10032
10033
10033
10033
10034
10034
10034
10035
10035
10035
10036
10036
10036
10037
10037
10037
10038
10038
10038
10039
10039
10039
10040
10040
10040
10041
10041
10041
10042
10042
10042
10043
10043
10043
10044
10044
10044
10045
10045
10045
10046
10046
10046
10047
10047
10047
10048
10048
10048
10049
10049
10049
finish


In [47]:
#sigo
print('sigo -----------------')
for seed in seed_list:
        sigo_delay_list = []
        for node in nodenum_list:
            if(node not in sigo_dict[seed]):
                print("seed", seed, 'node_num', node)

sigo -----------------


In [48]:
evaluation_path + 'Performance_evaluation_5_30.csv'

'../../get_data/13.500000_1.400000/1.500000Performance_evaluation_5_30.csv'